In [2]:
import json

import requests

from src.data_models import TextChunks


In [3]:
LLAMA_SERVER_URL = "http://localhost:32000"

In [4]:
system_prompt = """
## Money Stuff — Question Generation from Text Chunks

### Role
You are an expert at writing high-quality, chunk-grounded questions for a newsletter knowledge base.

You are given a JSON object containing a list of text chunks. For each chunk, you will create one or more questions that accurately reflect the key points, claims, definitions, mechanisms, examples, or implications in that chunk.

These questions will be used later for retrieval, study prompts, and search over the Money Stuff newsletter corpus.

---

### Input format
You will receive input shaped like:

{
  "chunks": [
    {"id": 1, "chunk": "..."},
    {"id": 2, "chunk": "..."}
  ]
}

---

### Output format (strict)
Return only JSON (no markdown, no commentary) in this shape:

{
  "questions": [
    {"chunk_id": 1, "questions": ["...", "..."]},
    {"chunk_id": 2, "questions": ["..."]}
  ]
}

Rules:
- Each questions array must contain 1–5 questions (prefer 2–4 when the chunk is dense).
- Keep questions answerable using only the chunk.
- Do not invent facts, numbers, names, or sources not present in the chunk.
- Do not summarize the chunk; write questions that target what the chunk actually says.
- Preserve the chunk’s meaning and nuance; avoid leading or loaded phrasing.
- If a chunk includes a quote or cited article, you may ask questions about what the quote or article claims, but stay grounded in the provided text.

---

### What makes a good question
Prefer questions that:
- Extract the core claim (for example: What is the author arguing about X?)
- Capture definitions (for example: What does the author mean by “AI rollups”?)
- Ask about mechanisms (for example: How does leverage create upside and downside asymmetry?)
- Contrast concepts (for example: How does VC’s “magic technology” differ from PE’s?)
- Identify motivations (for example: Why might VCs use rollups now?)
- Pull out concrete examples mentioned in the chunk

Avoid:
- Trivia, unless the chunk is explicitly about that detail
- Vague prompts such as “What is this about?”
- Multi-part questions that are hard to answer cleanly
- Questions that require external context or follow-up knowledge

---

### Style constraints
- Write concise, clear questions.
- Use the language and terms from the chunk when helpful (for example: “roll-up,” “non-recourse debt,” “AI agents”).
- Use question marks.
- Do not exceed roughly 25 words per question unless needed for precision.

---

### Examples

Example input:
{
  "chunks": [
    {
      "id": 1,
      "chunk": "# AI rollup\\n\\nPeople have been worried... We have talked a few times about “AI rollups,” where a venture capital firm buys a bunch of small companies, combines them, and sprinkles them with artificial intelligence."
    }
  ]
}

Example output:
{
  "questions": [
    {
      "chunk_id": 1,
      "questions": [
        "What is an “AI rollup” as described in this chunk?",
        "How does the chunk contrast traditional venture capital with the newer rollup approach?",
        "What kinds of local businesses does the chunk suggest could be targets of AI rollups?"
      ]
    }
  ]
}

---

Example input:
{
  "chunks": [
    {
      "id": 2,
      "chunk": "One way to think about it is that each of PE and VC has a powerful general-purpose technology..."
    }
  ]
}

Example output:
{
  "questions": [
    {
      "chunk_id": 2,
      "questions": [
        "What does the chunk describe as private equity’s “magic technology,” and how does it work?",
        "What does the chunk describe as venture capital’s “magic technology,” and what cost reductions does it imply?",
        "How does the chunk describe the risk and return profile of using leverage in a rollup strategy?",
        "According to the chunk, what is the longer-term profit opportunity after replacing back-office roles with AI?"
      ]
    }
  ]
}

---

Example input:
{
  "chunks": [
    {
      "id": 3,
      "chunk": "Here’s a Financial Times story about AI rollups: Top venture capital firms are borrowing a strategy..."
    }
  ]
}

Example output:
{
  "questions": [
    {
      "chunk_id": 3,
      "questions": [
        "According to the quoted Financial Times passage, what strategy are top VCs borrowing from private equity?",
        "What types of industries does the quote describe as suited to roll-up strategies, and why?",
        "What does the quote suggest is motivating VCs to pursue rollups right now?",
        "How does the quote contrast private equity rollups with VC claims about efficiency improvements?",
        "What is Savvy using AI to do, and what does that illustrate about the rollup thesis?"
      ]
    }
  ]
}

---

### Final reminder
Produce questions for every chunk in the input, using the output format exactly.
"""

In [1]:
chunks = json.load("data/text/money_stuff_chunked.md")
chunks

NameError: name 'json' is not defined

In [5]:
def count_tokens(prompt: str) -> int:
    try:
        response = requests.post(
            url=f"{LLAMA_SERVER_URL}/tokenize",
            headers={"Content-Type": "application/json"},
            data=json.dumps(
                {
                    "content": prompt,
                }
            ),
        )
        response.raise_for_status()

        response_json = response.json()
        tokens = response_json.get("tokens", [])

        return len(tokens)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return -1

system_prompt_tokens = count_tokens(system_prompt)
extracted_text_tokens = count_tokens(extracted_text)
total_tokens = system_prompt_tokens + extracted_text_tokens

print(f"System prompt tokens  : {system_prompt_tokens}")
print(f"Extracted text tokens : {extracted_text_tokens}")
print(f"Total tokens          : {total_tokens}")

System prompt tokens  : 868
Extracted text tokens : 1548
Total tokens          : 2416


In [6]:
def chunk_text(system_prompt, user_prompt, text):
    try:
        messages = [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_prompt + "\n\n" + text,
            },
        ]

        payload = {
            "messages": messages,
            "temperature": 0,
            "response_format": {
                "type": "json_object",
                "schema": TextChunks.model_json_schema(),
            },
        }

        response = requests.post(
            url=f"{LLAMA_SERVER_URL}/v1/chat/completions",
            data=json.dumps(payload),
        )
        response.raise_for_status()

        content = response.json()["choices"][0]["message"]["content"]
        return TextChunks.model_validate_json(content)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [7]:
user_prompt = """Below is the extracted text from a document.

Your task is to read the text carefully and split it into **coherent, topic-focused chunks**.

Rules:
- Each chunk must focus on **one clear topic or idea**.
- Do **not** summarize, paraphrase, rewrite, or modify the text in any way.
- Do **not** skip or omit any content. Every character must appear in exactly one chunk.
- Preserve the **exact text and formatting** as it appears, including:
  - Paragraph breaks
  - Lists and bullet points
  - Punctuation and capitalization
  - Quotes and emphasis
- Each chunk must be **self-contained and semantically meaningful** on its own.
- Do not split sentences, arguments, examples, or lists across chunks unless the topic clearly changes.

Return your output **exactly** in the format specified by the provided schema.
Do not include any explanations or text outside the JSON object.

EXTRACTED TEXT:
"""
result = chunk_text(system_prompt, user_prompt, extracted_text)

In [8]:
open("data/text/money_stuff_chunked.json", "w").write(result.model_dump_json(indent=2))

7845